In [ ]:
lines = """চলি চলো চলিস চলে চলেন
চলছি চলছ চলছিস চলছে চলছেন
চলেছই চলেছও চলেছিস চলেছে চলেছেন
চলো চলিস চলুক চলুন
চললাম চললে চললি চললে চললেন
চলছিলাম চলছিলে চলছিলি চলছিল চলছিলেন
চলেছিলাম চলেছিলে চলেছিলি চলেছিল চলেছিলেন
চলতাম চলতে চলতি চলত চলতেন
চলব চলবে চলবি চলবেন"""

list_of_lists = [line.split() for line in lines.split('\n')]

print(list_of_lists)


In [ ]:
original_list = [['করি', 'করো', 'করিস', 'করে', 'করেন'],
['করছি', 'করছ', 'করছিস', 'করছে', 'করছেন'],
['করেছি', 'করেছ', 'করেছিস', 'করেছে', 'করেছেন'],
['করো', 'করিস', 'করুক', 'করুন'],
['করলাম', 'করলে', 'করলি', 'করল', 'করলেন'],
['করছিলাম', 'করছিলে', 'করছিলি', 'করছিল', 'করছিলেন'],
['করেছিলাম', 'করেছিলে', 'করেছিলি', 'করেছিল', 'করেছিলেন'],
['করতাম', 'করতে', 'করতি', 'করত', 'করতেন'],
['করব', 'করবে', 'করবি', 'করবেন']]

pairs = []
for l in range(len(original_list[0])):
  for r in range(l+1, len(original_list[0])):
    pairs.extend([[lst[0], lst[1]] for lst in original_list])

print(pairs)

In [ ]:
puzzle = '''
পূর্বে আগে
পূর্বেই আগেই
অদ্য আজ
সহিত সাথে
দিয়া দিয়ে
অদ্য	আজ
অদ্যাপি	আজও
তথাপি	তবুও
নতুবা	নইলে
নতুবা নাহলে
প্রায়শ প্রায়ই
যদ্যপি	যদিও
কুত্রাপি	কোথাও
কিঞ্চিৎ	কিছু
কিঞ্চিৎ কিছুটা
'''

puzzle_list = puzzle.split('\n')


In [ ]:
puzzle_list = [i.strip() for i in puzzle_list]
puzzle_list = [i.rstrip() for i in puzzle_list]
puzzle_list = list(filter(None, puzzle_list))
print(puzzle_list)

In [ ]:
res = []
for p in puzzle_list:
  words = p.split()
  r_p = ' '.join(words)

  if r_p not in res:
    res.append(r_p)
print(res)

#Total Count of Bangla NLP Dataset

#Filter Mikolov Translate File

In [ ]:
#See if any word has not been
import re

# Set the filename for the input and output files
input_filename = 'Mikolov.txt'
output_filename = 'Mikolov-Filtered.txt'

# Set the regular expression pattern to match English words
english_word_pattern = re.compile(r'\b[a-zA-Z]+\b')

# Open the input and output files
with open(input_filename, 'r') as input_file, open(output_filename, 'w') as output_file:
    # Loop through each line in the input file
    for line in input_file:
        # Check if the line contains an English word
        if not re.search(english_word_pattern, line):
            # Write the line to the output file if it doesn't contain an English word
            output_file.write(line)

In [ ]:
#Remove phrases or more than 4 words
with open('Mikolov-Filtered.txt', 'r') as infile, open('Mikolov-Translate-Filtered_2.txt', 'w') as outfile:
    for line in infile:
        if len(line.split()) == 4:
            outfile.write(line)

In [ ]:
#Remove Duplicates
with open('Mikolov-Translate-Filtered_2.txt', 'r') as input_file, open('Mikolov-Translate-Filtered_3.txt', 'w') as output_file:
    for line in input_file:
        words = line.strip().split()
        if len(set(words)) == len(words):
            output_file.write(' '.join(words))
            output_file.write('\n')


#Testing Word Embedding

In [ ]:
!pip install transformers

In [ ]:
# https://discuss.huggingface.co/t/generate-raw-word-embeddings-using-transformer-models-like-bert-for-downstream-process/2958
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_word_idx(sent: str, word: str):
    return sent.split(" ").index(word)

def get_hidden_states(encoded, token_ids_word, model, layers):
     """Push input IDs through model. Stack and sum `layers` (last four by default).
        Select only those subword token outputs that belong to our word of interest
        and average them."""
     with torch.no_grad():
         output = model(**encoded)

     # Get all hidden states
     states = output.hidden_states
     # Stack and sum all requested layers
     output = torch.stack([states[i] for i in layers]).sum(0).squeeze()
     # Only select the tokens that constitute the requested word
     word_tokens_output = output[token_ids_word]

     return word_tokens_output.mean(dim=0)


def get_word_vector(sent, idx, tokenizer, model, layers):
     """Get a word vector by first tokenizing the input sentence, getting all token idxs
        that make up the word of interest, and then `get_hidden_states`."""
     encoded = tokenizer.encode_plus(sent, return_tensors="pt")
     # get all token idxs that belong to the word of interest
     token_ids_word = np.where(np.array(encoded.word_ids()) == idx)

     return get_hidden_states(encoded, token_ids_word, model, layers)


def get_embedding(layers=None, str_=""):
     # Use last four layers by default
     layers = [-4, -3, -2, -1] if layers is None else layers
     tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
     model = AutoModel.from_pretrained("bert-base-uncased", output_hidden_states=True)

     embeddings = []

     for idx, w in enumerate(str_.split()):
        embeddings.append(get_word_vector(str_, idx, tokenizer, model, layers))

     return torch.stack(embeddings).to(device)


if __name__ == '__main__':
     embd = get_embedding(str_ = "I like river side")
     print(embd.shape)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# create two matrices of vectors
A = torch.randn(10, 5)  # 10 vectors of length 5
B = torch.randn(8, 5)   # 8 vectors of length 5

# compute the cosine similarity between the two matrices
similarity_matrix = cosine_similarity(A, B)

# the similarity matrix is of shape (10, 8), where each element [i, j] is the
# cosine similarity between vector i in A and vector j in B
print(similarity_matrix.shape)

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

# text = "Special counsel Robert Mueller's team, Trump campaign officials, Jared Kushner"
text = "Special Counsel Robert Mueller, Trump campaign officials, former campaign manager Paul Manafort, former campaign official George Papadopoulos, President Trump, Attorney General Jeff Sessions, federal officials."

doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
!pip install -U bnlp_toolkit

In [ ]:
from bnlp import BengaliGlove

# Load the BengaliGlove embeddings
glove = BengaliGlove()

# Path to the embeddings file
embedding_file = glove.closest_word

# Create a dictionary to hold the word vectors
word_dict = {}

# Iterate over the embeddings file
with open(embedding_file, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.strip().split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        word_dict[word] = vector

# Print the length of the word dictionary
print(len(word_dict))

In [ ]:
python -m spacy download en_core_web_md

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("setu4993/LaBSE-Bengali")
model = AutoModel.from_pretrained("setu4993/LaBSE-Bengali")
sentence = "আমি ভাত খাই।"
input_ids = torch.tensor(tokenizer.encode(sentence)).unsqueeze(0)

with torch.no_grad():
    outputs = model(input_ids=input_ids)
    embeddings = outputs.last_hidden_state.mean(dim=1)

word_embeddings = embeddings.squeeze(0).split(1)
word_embeddings = [we.squeeze(0) for we in word_embeddings]

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/LaBSE'
model = SentenceTransformer(model_name)

# Example Bangla sentence
sentence = 'আমি বাংলায় কথা বলছি।'

# Compute LaBSE embedding
embedding = model.encode(sentence)

print(embedding)


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

sentence = 'আমি'
tokens = tokenizer.tokenize(sentence)
input_ids = tokenizer.encode(sentence, return_tensors='pt')

with torch.no_grad():
    outputs = model(input_ids)
    last_hidden_states = outputs[0]

word_embeddings = []
for i, token in enumerate(tokens):
    embedding = last_hidden_states[0][i+1]
    word_embeddings.append(embedding)

print(len(word_embeddings))
print(tokenizer.get_vocab())
print(tokenizer.ge)

In [ ]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load the LaBSE model
model_name = 'sentence-transformers/LaBSE'
model = SentenceTransformer(model_name)

# Define the Mikolov-style word analogy
a = 'লম্বা'
b = 'চারদিকে'
c = 'দূরত্ব'

# Tokenize the words
a_tokens = model.tokenize(a)
b_tokens = model.tokenize(b)
c_tokens = model.tokenize(c)

# Encode the words
a_embedding = model.encode(a)
b_embedding = model.encode(b)
c_embedding = model.encode(c)

# Compute the vector difference
diff_vector = b_embedding - a_embedding + c_embedding

# Find the closest word to the difference vector
words = ['লম্বা', 'চারদিকে', 'দূরত্ব']
word_embeddings = model.encode(words)
similarity = util.pytorch_cos_sim(word_embeddings, diff_vector).cpu().detach().numpy().flatten()
most_similar_word = words[np.argmax(similarity)]

print(f'{a} is to {b} as {c} is to {most_similar_word}')

In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models

In [ ]:
import numpy as np
from laserembeddings import Laser

# Load the pre-trained Laser model
laser = Laser()

# Define the Mikolov-style word analogy
a = 'লম্বা'
b = 'চারদিকে'
c = 'দূরত্ব'

# Tokenize the words
a_tokens = a.split()
b_tokens = b.split()
c_tokens = c.split()

# Encode the words
a_embedding = laser.embed_sentences([a_tokens], lang='bn')[0]
b_embedding = laser.embed_sentences([b_tokens], lang='bn')[0]
c_embedding = laser.embed_sentences([c_tokens], lang='bn')[0]

# Compute the vector difference
diff_vector = b_embedding - a_embedding + c_embedding

# Find the closest word to the difference vector
words = ['লম্বা', 'চারদিকে', 'দূরত্ব']
word_embeddings = laser.embed_sentences([words], lang='bn')[0]
similarity = np.dot(word_embeddings, diff_vector) / (np.linalg.norm(word_embeddings) * np.linalg.norm(diff_vector))
most_similar_word = words[np.argmax(similarity)]

print(f'{a} is to {b} as {c} is to {most_similar_word}')


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# Load the BanglaBERT model and tokenizer
model_name = 'csebuetnlp/banglabert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define the Mikolov-style word analogy
a = 'লম্বা'
b = 'চারদিকে'
c = 'দূরত্ব'

# Tokenize the words
a_tokens = tokenizer.tokenize(a)
b_tokens = tokenizer.tokenize(b)
c_tokens = tokenizer.tokenize(c)

# Convert the tokens to IDs
a_ids = tokenizer.convert_tokens_to_ids(a_tokens)
b_ids = tokenizer.convert_tokens_to_ids(b_tokens)
c_ids = tokenizer.convert_tokens_to_ids(c_tokens)

# Convert the IDs to PyTorch tensors
a_tensor = torch.tensor(a_ids).unsqueeze(0)
b_tensor = torch.tensor(b_ids).unsqueeze(0)
c_tensor = torch.tensor(c_ids).unsqueeze(0)

# Encode the words
with torch.no_grad():
    a_embedding = model(a_tensor)[0].mean(1)
    b_embedding = model(b_tensor)[0].mean(1)
    c_embedding = model(c_tensor)[0].mean(1)

# Compute the vector difference
diff_vector = b_embedding - a_embedding + c_embedding

# Find the closest word to the difference vector
words = ['লম্বা', 'চারদিকে', 'দূরত্ব']
word_ids = tokenizer.convert_tokens_to_ids(words)
word_tensor = torch.tensor(word_ids).unsqueeze(0)
with torch.no_grad():
    word_embeddings = model(word_tensor)[0].mean(1)
similarity = np.dot(word_embeddings, diff_vector.T) / (np.linalg.norm(word_embeddings, axis=1) * np.linalg.norm(diff_vector))
most_similar_word = words[np.argmax(similarity)]

print(f'{a} is to {b} as {c} is to {most_similar_word}')


In [ ]:
!pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModel
import numpy as np
import torch

# Load the pre-trained BanglaT5 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglat5")
model = AutoModel.from_pretrained("csebuetnlp/banglat5")

# Define the Mikolov-style word analogy
a = 'লম্বা'
b = 'চারদিকে'
c = 'দূরত্ব'

# Tokenize the words
a_tokens = tokenizer.tokenize(a)
b_tokens = tokenizer.tokenize(b)
c_tokens = tokenizer.tokenize(c)

# Encode the words
with torch.no_grad():
    a_embedding = model(torch.tensor(tokenizer.encode(a)).unsqueeze(0))[0][0].numpy()
    b_embedding = model(torch.tensor(tokenizer.encode(b)).unsqueeze(0))[0][0].numpy()
    c_embedding = model(torch.tensor(tokenizer.encode(c)).unsqueeze(0))[0][0].numpy()

# Compute the vector difference
diff_vector = b_embedding - a_embedding + c_embedding

# Find the closest word to the difference vector
words = ['লম্বা', 'চারদিকে', 'দূরত্ব']
word_embeddings = np.zeros((len(words), len(a_embedding)))
for i, word in enumerate(words):
    with torch.no_grad():
        embedding = model(torch.tensor(tokenizer.encode(word)).unsqueeze(0))[0][0].numpy()
        word_embeddings[i] = embedding

similarity = np.dot(word_embeddings, diff_vector) / (np.linalg.norm(word_embeddings) * np.linalg.norm(diff_vector))
most_similar_word = words[np.argmax(similarity)]

print(f'{a} is to {b} as {c} is to {most_similar_word}')


In [ ]:
!pip install laserembeddings

In [ ]:
!python -m laserembeddings download-models

In [ ]:
import numpy as np
import torch
from laserembeddings import Laser

laser = Laser()

sentence = "আমি বাংলায় কথা বলি"
embedding = laser.embed_sentences([sentence], lang='bn')[0]

embedding = np.array(embedding)
print(embedding)

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = AutoModel.from_pretrained("sentence-transformers/LaBSE")

sentences = [
    "আমি বাংলায় কথা বলি",
    "আমার দেশ বাংলাদেশ"
]

encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    model_output = model(**encoded_input)
    sentence_embeddings = model_output.last_hidden_state[:, 0, :].numpy()

print(sentence_embeddings)


In [ ]:
my_list = ['hello', 'world', 'how', 'are', 'you']
with open('filename.txt', 'w') as file:
    file.write(str(my_list))

#Data Format for Table and Graph in Paper

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
models = ['Mikolov-Filtered', 'Our Dataset (Overall)']
embeddings = ['Word2Vec', 'GloVe', 'fastText','LABSE', 'bnBERT', 'LASER',  'bnBART']
mikolov_data = [0.9, 1.1, 0.3,  4.4, 0.0, 1.6,  1.6]
our_data = [0.3, 0.2, 0.6,  6.2, 0.1, 1.1, 11.9]

# Set the figure size
plt.figure(figsize=(8, 4))

# Define colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Define patterns for the bars
patterns = cycle(['\\', '*','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], mikolov_data, width=0.2, hatch=next(patterns), label='Mikolov-trans. Dataset',color=colors[2])
plt.bar([e + 0.2 for e in range(len(embeddings))], our_data, width=0.2, hatch=next(patterns), label='Our Dataset', color=colors[5])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.xticks([e + 0.2 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14)

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('comparison_top_1.pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Data
models = ['Mikolov-Filtered', 'Our Dataset (Overall)']
embeddings = ['Word2Vec', 'GloVe', 'fastText',  'LABSE', 'bnBERT', 'LASER',  'bnBART']
mikolov_data = [9.3, 10.0, 0.9,  10.6, 0.0,  6.8, 3.5]
our_data = [8.2, 6.3, 3.0,  18.0, 0.4, 10.0,  15.8]

# Set the figure size
plt.figure(figsize=(8, 4))

# Define colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Define patterns for the bars
patterns = cycle(['\\', '*','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], mikolov_data, width=0.2, hatch=next(patterns), label='Mikolov-trans. Dataset',color=colors[2])
plt.bar([e + 0.2 for e in range(len(embeddings))], our_data, width=0.2, hatch=next(patterns), label='Our Dataset', color=colors[5])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.xticks([e + 0.2 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14)

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('comparison_top_5.pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings = ['Word2Vec', 'GloVe', 'fastText',  'LABSE', 'bnBERT', 'LASER', 'bnBART']
top1 = [0.5, 0.2, 0.1,  3.8, 0.1, 0.4, 2.8]
top5 = [9.0, 7.7, 1.3,  12.8, 0.4, 5.4,  5.4]

# Set the figure size
plt.figure(figsize=(8, 4))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']

# Define patterns for the bars
patterns = cycle(['\o', '+/','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], top1, width=0.2, hatch=next(patterns), label='Top-1')
plt.bar([e + 0.2 for e in range(len(embeddings))], top5, width=0.2, hatch=next(patterns), label='Top-5')

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
# plt.title('Semantic Accuracy by Embedding', fontsize=14)
plt.xticks([e + 0.1 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14, loc='upper left')

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('semantic_top_1_5.pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings = ['Word2Vec', 'GloVe', 'fastText',  'LABSE', 'bnBERT', 'LASER',  'bnBART']
top1 = [0.1, 0.1, 1.0,8.5, 0.0, 1.7,  21.0]
top5 = [7.3, 4.9, 4.6,  23.1, 0.3, 14.5,  26.1]

# Set the figure size
plt.figure(figsize=(8, 4))

# Define patterns for the bars
patterns = cycle(['\o', '+/','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], top1, width=0.2, hatch=next(patterns), label='Top-1')
plt.bar([e + 0.2 for e in range(len(embeddings))], top5, width=0.2, hatch=next(patterns), label='Top-5')

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
# plt.title('Syntactic Accuracy by Embedding', fontsize=14)
plt.xticks([e + 0.1 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14, loc='upper left')

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('syntactic_top_1_5.pdf', bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings =['Word2Vec', 'GloVe', 'fastText',  'LaBSE', 'bnBERT', 'LASER','bnBART']
top3 = [5.8,4.2,0.5,9.4,0.2,3.4,4.6]
top10 = [14.8,13.0,2.7,16.4,0.6,8.2,6.5]
# Set the figure size
plt.figure(figsize=(8, 4))

# Define patterns for the bars
patterns = cycle(['/.','\o', '+/','\\', '.', '*', '/'])

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#000000']


# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], top3, width=0.2, hatch=next(patterns), label='Top-3', color = colors[-3])
plt.bar([e + 0.2 for e in range(len(embeddings))], top10, width=0.2, hatch=next(patterns), label='Top-10',color = colors[-2])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
# plt.title('Semantic Accuracy by Embedding', fontsize=14)
plt.xticks([e + 0.1 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14, loc='upper right')

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('semantic_top_3_10.pdf', bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings =['Word2Vec', 'GloVe', 'fastText',  'LaBSE', 'bnBERT', 'LASER','bnBART']
top3 = [4.4,3.3,2.9,19.2,0.2,11.9,24.5]
top10 = [9.3,6.2,7.3,28.2,0.4,19.3,27.6]
# Set the figure size
plt.figure(figsize=(8, 4))

# Define patterns for the bars
patterns = cycle(['/.','\o', '+/','\\', '.', '*', '/'])

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#000000']


# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], top3, width=0.2, hatch=next(patterns), label='Top-3', color = colors[-3])
plt.bar([e + 0.2 for e in range(len(embeddings))], top10, width=0.2, hatch=next(patterns), label='Top-10',color = colors[-2])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
# plt.title('Semantic Accuracy by Embedding', fontsize=14)
plt.xticks([e + 0.1 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14, loc='upper left')

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('syntactic_top_3_10.pdf', bbox_inches='tight')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
models = ['Mikolov-Filtered', 'Our Dataset (Overall)']
embeddings = ['Word2Vec','GloVe','fastText',  'LaBSE', 'bnBERT', 'LASER','bnBART']
mikolov_data = [6.1,6.6,0.6,8.6,0.0,4.9,2.8]
our_data = [5.1,3.8,1.7,14.3,0.2,7.7,14.6]

# Set the figure size
plt.figure(figsize=(8, 4))

# Define colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#000000']


# Define patterns for the bars
patterns = cycle(['\\', '*','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], mikolov_data, width=0.2, hatch=next(patterns), label='Mikolov-trans. Dataset',color=colors[-5])
plt.bar([e + 0.2 for e in range(len(embeddings))], our_data, width=0.2, hatch=next(patterns), label='Our Dataset', color=colors[-10])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.xticks([e + 0.2 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14)

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('comparison_top_3.pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
models = ['Mikolov-Filtered', 'Our Dataset (Overall)']
embeddings = ['Word2Vec','GloVe', 'fastText', 'LaBSE', 'bnBERT', 'LASER','bnBART']
mikolov_data = [15.2, 15.0, 1.7, 13.1, 0.1, 10.1, 4.4]
our_data = [12.1, 9.6, 5.0, 22.3, 0.5, 13.8, 17.1]

# Set the figure size
plt.figure(figsize=(8, 4))

# Define colors for the bars
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#000000']


# Define patterns for the bars
patterns = cycle(['\\', '*','\\', '.', '*', '/'])

# Plot the bar chart
plt.bar([e for e in range(len(embeddings))], mikolov_data, width=0.2, hatch=next(patterns), label='Mikolov-trans. Dataset',color=colors[-5])
plt.bar([e + 0.2 for e in range(len(embeddings))], our_data, width=0.2, hatch=next(patterns), label='Our Dataset', color=colors[-10])

# Add labels and titles
plt.xlabel('Embedding', fontsize=14)
plt.ylabel('Accuracy (%)', fontsize=14)
plt.xticks([e + 0.2 for e in range(len(embeddings))], embeddings, fontsize=13)
plt.yticks(fontsize=13)
plt.legend(fontsize=14)

# Adjust spacing
plt.subplots_adjust(bottom=0.15)

# Save the figure
plt.savefig('comparison_top_10.pdf', bbox_inches='tight')

# Show the plot
plt.show()


#Merge Graph

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings = ['Word2Vec', 'GloVe', 'fastText', 'LABSE', 'bnBERT', 'LASER', 'bnBART']
top1 = [0.5, 0.2, 0.1, 3.8, 0.1, 0.4, 2.8]
top3 = [5.8, 4.2, 0.5, 9.4, 0.2, 3.4, 4.6]
top5 = [9.0, 7.7, 1.3, 12.8, 0.4, 5.4, 5.4]
top10 = [14.8, 13.0, 2.7, 16.4, 0.6, 8.2, 6.5]

# Set the figure size
plt.figure(figsize=(10, 6))

# Define patterns for the bars
patterns = cycle(['\o', '+/', '\\', '*', '*', '/'])

# Define colors for the bars
# colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#d62728', '#9467bd', '#8c564b', '#e377c2']
colors = ['#92c6df', '#ffa97e', '#97d7a7', '#808080', '#eb9f9f', '#bfa0d4', '#c3a697', '#f0bdd6']

# Plot the bar chart for Top-1
plt.bar([e for e in range(len(embeddings))], top1, width=0.15, hatch=next(patterns), label='Top-1', color=colors[0])
# Plot the bar chart for Top-3
plt.bar([e + 0.15 for e in range(len(embeddings))], top3, width=0.15, hatch=next(patterns), label='Top-3', color=colors[1])
# Plot the bar chart for Top-5
plt.bar([e + 0.3 for e in range(len(embeddings))], top5, width=0.15, hatch=next(patterns), label='Top-5', color=colors[2])
# Plot the bar chart for Top-10
plt.bar([e + 0.45 for e in range(len(embeddings))], top10, width=0.15, hatch=next(patterns), label='Top-10', color=colors[3])

# Add labels and titles
plt.xlabel('Embedding', fontsize=18)
plt.ylabel('Accuracy (%)', fontsize=18)
plt.xticks([e + 0.225 for e in range(len(embeddings))], embeddings, fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=16, loc='upper right')

# Save the figure
plt.savefig('semantic_all.pdf', bbox_inches='tight')

# Show the plot
plt.show()


# Compute Average

In [ ]:
import matplotlib.pyplot as plt
from itertools import cycle

# Data
embeddings = ['Word2Vec', 'GloVe', 'fastText', 'LABSE', 'bnBERT', 'LASER', 'bnBART']
top1 = [0.1, 0.1, 1.0, 8.5, 0.0, 1.7, 21.0]
top3 = [4.4, 3.3, 2.9, 19.2, 0.2, 11.9, 24.5]
top5 = [7.3, 4.9, 4.6, 23.1, 0.3, 14.5, 26.1]
top10 = [9.3, 6.2, 7.3, 28.2, 0.4, 19.3, 27.6]

# Set the figure size
plt.figure(figsize=(10, 6))

# Define patterns for the bars
patterns = cycle(['\o', '+/', '\\', '*', '.', '/'])

# Define colors for the bars
# colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2']
colors = ['#92c6df', '#ffa97e', '#97d7a7', '#808080', '#eb9f9f', '#bfa0d4', '#c3a697', '#f0bdd6']


# Plot the bar chart for Top-1
plt.bar([e for e in range(len(embeddings))], top1, width=0.15, hatch=next(patterns), label='Top-1', color=colors[0])
# Plot the bar chart for Top-3
plt.bar([e + 0.15 for e in range(len(embeddings))], top3, width=0.15, hatch=next(patterns), label='Top-3', color=colors[1])
# Plot the bar chart for Top-5
plt.bar([e + 0.3 for e in range(len(embeddings))], top5, width=0.15, hatch=next(patterns), label='Top-5', color=colors[2])
# Plot the bar chart for Top-10
plt.bar([e + 0.45 for e in range(len(embeddings))], top10, width=0.15, hatch=next(patterns), label='Top-10', color=colors[3])

# Add labels and titles
plt.xlabel('Embedding', fontsize=18)
plt.ylabel('Accuracy (%)', fontsize=18)
plt.xticks([e + 0.225 for e in range(len(embeddings))], embeddings, fontsize=18)
plt.yticks(fontsize=18)
plt.legend(fontsize=16, loc='upper left')

# Save the figure
plt.savefig('syntactic_all.pdf', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
text = """fasttext	0.1	1.3	0.5	2.7
word2vec	0.5	9.0	5.8	14.8
glove	0.2	7.7	4.2	13.0
laser	0.4	5.4	3.4	8.2
labse	3.8	12.8	9.4	16.4
bnBERT	0.1	0.4	0.2	0.6
bnTrans	2.8	5.4	4.6	6.5
fasttext	1.0	4.6	2.9	7.3
word2vec	0.1	7.3	4.4	9.3
glove	0.1	4.9	3.3	6.2
laser	1.7	14.5	11.9	19.3
labse	8.5	23.1	19.2	28.2
bnBERT	0.0	0.3	0.2	0.4
bnTrans	21.0	26.1	24.5	27.6
"""

lines = text.split('\n')
data = [line.split('\t') for line in lines]
data = [[row[0]] + [float(x) for x in row[1:]] for row in data]
print(data)

In [ ]:
import numpy as np
import pandas as pd


import statistics

data = data
res = []

embed = ['fasttext', 'word2vec', 'glove', 'laser', 'labse', 'bnBERT', 'bnTrans']

for e in embed:
  embed_data = [row[1:] for row in data if row[0] == e]
  avg = [statistics.mean(col) for col in zip(*embed_data)]

  res.append(avg)

# create a list of rows
rows = []
for i in range(len(embed)):
    row = [embed[i]] + res[i]
    rows.append(row)

columns = ['Model', 'Top-1', 'Top-3', 'Top-5','Top-10']
# create the dataframe
df = pd.DataFrame(rows, columns=columns)

df.to_excel('example.xlsx', index=False)
print(df)


Write text to excel file

In [ ]:
data = """
Affix_বিভক্তি chatgpt 26.32% 26.32% 26.32% 28.95%
Affix_বিভক্তি bard 10.53% 18.42% 21.05% 21.05%
Tense chatgpt 8.33% 11.81% 13.89% 17.36%
Tense bard 1.39% 4.17% 9.03% 11.11%
"""

lines = data.split("\n")
data_list = [line.split() for line in lines]



In [ ]:
import openpyxl

# Create a new workbook
workbook = openpyxl.Workbook()

# Select the active worksheet
worksheet = workbook.active

# Define the headers
headers = ['Task', 'Embedding', 'Top-1 (%)', 'Top-3 (%)', 'Top-5 (%)', 'Top-10 (%)']

# Write the headers to the first row
worksheet.append(headers)

# Write the data to the worksheet
data = data_list

for row in data:
    worksheet.append(row)

# Save the workbook
workbook.save('data.xlsx')


In [ ]:
!pip install -U bntransformer

In [ ]:
!pip install transformers

In [ ]:
from bntransformer import BanglaTokenizer
import transformers

model_path = "facebook/bart-large-cnn"

bntokenizer = BanglaTokenizer(model_path=model_path)

text = "আমি বাংলায় গান গাই ।"
tokens = bntokenizer.tokenize(text)
encode_ids = bntokenizer.encode(text)
print(encode_ids)

# # Get the word embeddings of each token using the pre-trained model
# word_embeddings = []
# for token_id in encode_ids:
#     token = bntokenizer.decode([token_id])
#     emb = model.get_word_vector(token)
#     word_embeddings.append(emb)

# print(word_embeddings)

In [ ]:
from bntransformer import BanglaTokenizer
import torch
import transformers

# Load the BART model
model_name = "facebook/bart-large-cnn"
model = transformers.BartModel.from_pretrained(model_name)

# Set the model to evaluation mode
model.eval()

# Load the BanglaTokenizer
tokenizer = BanglaTokenizer(model_path=model_name)

# Tokenize the input text
input_text = "আমি"
tokens = tokenizer.tokenize(input_text)
print(tokens)

# Convert the tokens to token IDs
input_ids = tokenizer.encode(input_text)

# Convert the input_ids to a PyTorch tensor
input_tensor = torch.tensor(input_ids)

# Generate the word embeddings
with torch.no_grad():
    outputs = model(input_tensor.unsqueeze(0))
    embeddings = outputs.last_hidden_state.squeeze(0)

embeddings = torch.mean(embeddings, dim=0)

print(embeddings.shape)
